<a href="https://colab.research.google.com/github/BetriuJaume/Projecte_Prog/blob/master/RandomForest_Code_and_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyreadstat
import pyreadstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 27.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.utils import resample
from imblearn.over_sampling import SMOTENC
!pip install h2o
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

h2o.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.0 MB 17 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=4e4bd68bf0d3d7211f16c6e7a2e2fba6e6d2567118c668395ee31a8954e5d4e5
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpinkfai8l
  JVM stdout: /tmp/tmpinkfai8l/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp

H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,1 month and 12 days
H2O_cluster_name:,H2O_from_python_unknownUser_k87w8h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
dades_orig=pd.read_spss('EarlyLifeCovidPACIENTES V6 ID.sav')
dades_extra=pd.read_spss('extra_data.sav')

dades_extra['Talla_mt'] = dades_extra['Talla']/100
dades_extra=dades_extra.rename(columns = {'Tabaco_Sino':'Tabaco_SIno'})
dades_extra=dades_extra.drop(list(set(dades_extra.columns)-set(dades_orig)),axis=1)
dades_extra['IUGR_missing']= [1.]*len(dades_extra)

for colu in dades_extra.columns[12:-2]:
    dades_extra[colu]=['si' if x==1 else 'no' for x in dades_extra[colu]]

dades=pd.concat([dades_orig, dades_extra], ignore_index=True)

def canvi_tab(elem):
  if(elem=='si (pasado o actual)'):
    return 'yes'
  if(elem=='nunca'):
    return 'no'
def canvi(elem):
  if(elem=='si'):
    return 'yes'
  if(elem=='no'):
    return 'no'
def canvi_UCI(elem):
  if(elem==1):
    return 'Yes'
  if(elem==0):
    return 'No'
def canvi_sex(elem):
  if(elem=='mujer'):
    return 'woman'
  if(elem=='hombre'):
    return 'man'

dades['Tabaco_SIno']=dades['Tabaco_SIno'].apply(canvi_tab)
dades['UCI']=dades['UCI'].apply(canvi_UCI)
for columna in dades.columns[14:]:
  dades[columna]=dades[columna].apply(canvi)
dades['Gender']=dades['Gender'].apply(canvi_sex)
dades['IUGR_calc']=dades['IUGR_calc'].apply(canvi_UCI)
dades.columns=pd.Index(['ID', 'UCI', 'IUGR_missing', 'Age', 'Gender', 'Size', 'Weight',
       'Size_mt', 'IMC', 'BW', 'BW_2500', 'percentil_birth', 'IUGR_calc',
       'Tobacco_yes_no', 'Hipertension', 'Heart_diseases', 'DM', 'Dyslipidemia',
       'Obesity', 'Kidney_disease', 'Autoimmune', 'Cancer', 'Thyroid', 'Infectious',
       'Psychiatric'])

dades['Age']=dades['Age'].fillna(dades['Age'].mean())
dades['Size']=dades['Size'].fillna(dades['Size'].mean())
dades['Weight']=dades['Weight'].fillna(dades['Weight'].mean())
dades['Size_mt']=dades['Size']/100
dades['IMC']=dades['IMC'].fillna(dades['IMC'].mean())
dades['percentil_birth']=dades['percentil_birth'].fillna(dades['percentil_birth'].mean())
dades['IUGR_calc']=dades['IUGR_calc'].fillna(dades['IUGR_calc'].mode().iloc[0])

In [ ]:
dades_train = dades.sample(frac=0.8, random_state=25).drop('ID',axis=1)
dades_test = dades.drop(dades_train.index).drop('ID',axis=1)

We will design a function to caluculate the accuraccy of the models:

In [ ]:
def precisions(reals,prediccions):
  cont_Yes=0
  cont_No=0
  for i in range(len(reals)):
    if (reals.iloc[i]==prediccions[i]) & (reals.iloc[i]=='Yes'):
      cont_Yes=cont_Yes+1
    if (reals.iloc[i]==prediccions[i]) & (reals.iloc[i]=='No'):
      cont_No=cont_No+1
  print('Precision for the Yes: '+str(cont_Yes/len(reals[reals=='Yes'])))
  print('Precision for the No: '+str(cont_No/len(reals[reals=='No'])))
  print('Total precision: '+str((cont_Yes+cont_No)/len(reals)))

Here we will create the upsampled training data to balance the dataset:

In [ ]:
dades_train_UCI_No=dades_train[dades_train['UCI']=='No']
dades_train_UCI_Yes=dades_train[dades_train['UCI']=='Yes']
dades_UCI_Yes_upsampled=resample(dades_train_UCI_Yes,replace=True,n_samples=len(dades_train_UCI_No),random_state=123)
dades_train_upsampled=pd.concat([dades_train_UCI_No,dades_UCI_Yes_upsampled])

 and here we will create the training data balanced with SMOTENC

In [ ]:
categ_indexes=[0,2,7,8]+list(range(10,dades_train.shape[1]-1))
smot = SMOTENC(categorical_features=categ_indexes, random_state=123)
dades_train_SMOTENC, dades_train_SMOTENC_UCI = smot.fit_resample(dades_train.drop('UCI',axis=1), dades_train['UCI'])
dades_train_SMOTENC['UCI']=dades_train_SMOTENC_UCI

#Training of the Random Forest models from the library H2O

We will use the upsampling training dataset for training one model, the SMOTENC training dataset for training another model and then we will compare them

In [ ]:
dades_train_upsampled_h2o=h2o.H2OFrame(dades_train_upsampled)
dades_train_SMOTENC_h2o=h2o.H2OFrame(dades_train_SMOTENC)
predictors_upsampling=['BW_2500','Cancer','Gender','Hipertension','Heart_diseases','Infectious','Obesity','Thyroid','Age','IMC','Size','Weight','percentil_birth']   #h2o.H2OFrame(dades_train_upsampled).drop('UCI',axis=1).columns
predictors_SMOTENC=['BW_2500','Cancer','DM','Dyslipidemia','Gender','Hipertension','Obesity','Psychiatric','Age']
response='UCI'

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
model_random_forest_upsampling = H2ORandomForestEstimator(ntrees=500,
                                    max_depth=5,
                                    min_rows=20,
                                    seed=123,
                                    mtries=-1)
model_random_forest_SMOTENC = H2ORandomForestEstimator(ntrees=500,
                                    max_depth=20,
                                    min_rows=10,
                                    seed=123,
                                    mtries=-1)

model_random_forest_upsampling=model_random_forest_upsampling.train(x=[e for e in dades_train_upsampled_h2o.columns if e not in ['IUGR_missing','UCI','Size','BW']],
                                                         y=response,
                                                         training_frame=dades_train_upsampled_h2o)    #dades_train_upsampled_h2o)
model_random_forest_SMOTENC=model_random_forest_SMOTENC.train(x=predictors_SMOTENC,
                                                      y=response,
                                                      training_frame=dades_train_SMOTENC_h2o)

prediccions_random_forest_upsampling=model_random_forest_upsampling.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))
prediccions_random_forest_SMOTENC=model_random_forest_SMOTENC.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


Once we have both models trained we can calculate our predictions

In [ ]:
prediccions_random_forest_upsampling=model_random_forest_upsampling.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))
prediccions_random_forest_SMOTENC=model_random_forest_SMOTENC.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


And check the accuracy

In [ ]:
prediccions_random_forest_upsampling= prediccions_random_forest_upsampling['predict'].as_data_frame()
prediccions_random_forest_upsampling=prediccions_random_forest_upsampling['predict'].tolist()

prediccions_random_forest_SMOTENC= prediccions_random_forest_SMOTENC['predict'].as_data_frame()
prediccions_random_forest_SMOTENC=prediccions_random_forest_SMOTENC['predict'].tolist()

print('Upsampling:')
precisions(dades_test['UCI'],prediccions_random_forest_upsampling)
print('___________________________________')
print('SMOTENC:')
precisions(dades_test['UCI'],prediccions_random_forest_SMOTENC)

Upsampling:
Precision for the Yes: 0.88
Precision for the No: 0.7129629629629629
Total precision: 0.7221006564551422
___________________________________
SMOTENC:
Precision for the Yes: 0.68
Precision for the No: 0.7314814814814815
Total precision: 0.7286652078774617


In [ ]:
dades_train_for_grid=h2o.H2OFrame(dades_train_upsampled)

params = {'ntrees': [250, 300, 350, 400, 450, 500, 550, 600],
          'max_depth': [16, 18, 20, 22, 24, 26, 28, 30],
          'min_rows': [16, 18, 20, 22, 24, 26, 28, 30],
          'mtries': [-1,3,5]}

params1 = {'ntrees': [500,600,700],
          'max_depth': [3,4,5,6,7],
          'min_rows': [6,10,15,20],
          'mtries': [-1]}

grid = H2OGridSearch(model = H2ORandomForestEstimator(seed = 123),
                     grid_id = 'gbm_grid1',
                     hyper_params = params1,
                     search_criteria = {'strategy': "Cartesian"})

grid_models=grid.train(x=[e for e in dades_train_for_grid.columns if e not in ['IUGR_missing','UCI','Size','BW']],
                                                         y=response,
                                                         training_frame=h2o.H2OFrame(dades_train_upsampled))

print(grid_models)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
      max_depth  min_rows  mtries  ntrees           model_ids   logloss
0           7.0       6.0    -1.0   600.0  gbm_grid1_model_25  0.323067
1           7.0       6.0    -1.0   700.0  gbm_grid1_model_45  0.323165
2           7.0       6.0    -1.0   500.0   gbm_grid1_model_5  0.323490
3           7.0      10.0    -1.0   500.0  gbm_grid1_model_10  0.337059
4           7.0      10.0    -1.0   600.0  gbm_grid1_model_30  0.337179
5           7.0      10.0    -1.0   700.0  gbm_grid1_model_50  0.337532
6           7.0      15.0    -1.0   500.0  gbm_grid1_model_15  0.355277
7           7.0      15.0    -1.0   600.0  gbm_grid1_model_35  0.355470
8           7.0      15.0    -1.0   700.0  gbm_grid1_model_55  0.35

In [ ]:
for model in grid_models:

  print(model.summary())
  prediccions = model.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))

  prediccions = prediccions['predict'].as_data_frame()
  prediccions = prediccions['predict'].tolist()

  print('Upsampling:')
  precisions(dades_test['UCI'],prediccions)
  print('___________________________________')


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,600.0,600.0,353841.0,7.0,7.0,7.0,25.0,57.0,42.261665



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Upsampling:
Precision for the Yes: 0.56
Precision for the No: 0.8472222222222222
Total precision: 0.8315098468271335
___________________________________

Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,700.0,700.0,412338.0,7.0,7.0,7.0,25.0,57.0,42.204285



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Upsampling:
Precision for the Yes: 0.56
Precision for the No: 0.8425925925925926
Total precision: 0.8271334792122538
___________________________________

Model Summary: 


...


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Upsampling:
Precision for the Yes: 0.96
Precision for the No: 0.7037037037037037
Total precision: 0.7177242888402626
___________________________________

Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,700.0,700.0,107995.0,3.0,3.0,3.0,4.0,8.0,7.57



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Upsampling:
Precision for the Yes: 0.96
Precision for the No: 0.6712962962962963
Total precision: 0.687089715536105
___________________________________

Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,500.0,500.0,76993.0,3.0,3.0,3.0,4.0,8.0,7.552



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Upsampling:
Precision for the Yes: 0.88
Precision for the No: 0.7222222222222222
Total precision: 0.7308533916849015
___________________________________


#Design, training and hiperparameter adjusting of the Neural Network

In [ ]:
import torch
import torch.utils.data as data_utils
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

We have to do a first codification of the categorical predictors because Torch tensors only work with categorical values

In [ ]:
def codGender(elem):
  if elem=='man':
    return 1.
  else:
    return 0.
def codUCI(elem):
  if elem=='Yes':
    return 1.
  else:
    return 0.
def codBW(elem):
  if elem=='BW normal':
    return 0.
  else:
    return 1.
def cod(elem):
  if elem=='yes':
    return 1.
  else:
    return 0.

dades_cod_train=dades_train_upsampled.drop('IUGR_missing',axis=1).copy()
dades_cod_test=dades_test.drop('IUGR_missing',axis=1).copy()

for df in [dades_cod_train,dades_cod_test]:
  df['UCI']=df['UCI'].apply(codUCI)
  df['Gender']=df['Gender'].apply(codGender)
  df['BW_2500']=df['BW_2500'].apply(codBW)
  df['IUGR_calc']=df['IUGR_calc'].apply(codUCI)
  for element in df.columns[11:]:
    df[element]=df[element].apply(cod)

target_train=torch.tensor(dades_cod_train['UCI'].values)
features_train=torch.tensor(dades_cod_train.drop('UCI',axis=1).values)
Dades_Tensor_train=TensorDataset(features_train,target_train)

target_test=torch.tensor(dades_cod_test['UCI'].values)
features_test=torch.tensor(dades_cod_test.drop('UCI',axis=1).values)
Dades_Tensor_test=TensorDataset(features_test,target_test)

from torch.utils.data import random_split

train_torch,val_torch=random_split(Dades_Tensor_train,[int(0.8*len(Dades_Tensor_train))+1,int(0.2*len(Dades_Tensor_train))],generator=torch.Generator().manual_seed(123))
test_torch=Dades_Tensor_test

train_torch1=DataLoader(train_torch,batch_size=10,shuffle=True)
val_torch1=DataLoader(val_torch,batch_size=5,shuffle=True)
test_torch1=DataLoader(test_torch,batch_size=5,shuffle=True)

We design our net

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_layer=nn.Linear(features_train.size()[1],14) #weights
    #self.hidden1=nn.Linear(14,7)
    #self.hidden2=nn.Linear(7,7)
    #self.hidden3=nn.Linear(7,10)
    #self.hidden4=nn.Linear(10,7)
    self.output=nn.Linear(14,2)
  
  def forward(self, data): #activation functions, we will use the ReLu function
    data=F.relu(self.input_layer(data))
    #data=F.relu(self.hidden1(data))
    #data=F.relu(self.hidden2(data))
    #data=F.relu(self.hidden3(data))
    #data=F.relu(self.hidden4(data))
    data=self.output(data)
    return F.log_softmax(data, dim=1)

 and we train it with the **upsampling training dataset** and once we have the NN trained we will use the validation dataset to adjust the hiperparameter **rep**:

In [ ]:
Accuracy_yes=[]
Accuracy_no=[]

network=Network()
learn_rate=optim.Adam(network.parameters(), lr=0.0001)
for rep in range(30):
  for i in range(rep):
    for data in train_torch1:
      predictors, objectiu=data
      network.zero_grad()
      result=network(predictors.float())
      loss=F.nll_loss(result,objectiu.long())  #loss=F.cross_entropy(result,objectiu.long())  # negative log likelihood loss
      loss.backward()
      learn_rate.step()
    #print(loss)

  network.eval() #we tell the network that we are not training anymore

  correct_No=0
  correct_Yes=0
  total_No=0
  total_Yes=0

  with torch.no_grad(): #validation set to adjust the hiperparameters
    for data in val_torch1:
      predictors, objectiu=data
      result=network(predictors.float())
      for index, tensor_value in enumerate(result):
        if(objectiu[index]==1):
          total_Yes=total_Yes+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_Yes=correct_Yes+1
        if(objectiu[index]==0):
          total_No=total_No+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_No=correct_No+1

  accuracy_Yes=correct_Yes/total_Yes
  accuracy_No=correct_No/total_No

  Accuracy_yes.append(accuracy_Yes)
  Accuracy_no.append(accuracy_No)

  print('Accuracy Yes '+str(accuracy_Yes)+' with rep='+str(rep))
  print('Accuracy No '+str(accuracy_No)+' with rep='+str(rep))
  print('_________________________________________________________')

Accuracy Yes 1.0 with rep=0
Accuracy No 0.0 with rep=0
_________________________________________________________
Accuracy Yes 0.6887608069164265 with rep=1
Accuracy No 0.5257142857142857 with rep=1
_________________________________________________________
Accuracy Yes 0.8559077809798271 with rep=2
Accuracy No 0.40285714285714286 with rep=2
_________________________________________________________
Accuracy Yes 0.5936599423631124 with rep=3
Accuracy No 0.6828571428571428 with rep=3
_________________________________________________________
Accuracy Yes 0.4610951008645533 with rep=4
Accuracy No 0.7685714285714286 with rep=4
_________________________________________________________
Accuracy Yes 0.8069164265129684 with rep=5
Accuracy No 0.58 with rep=5
_________________________________________________________
Accuracy Yes 0.5331412103746398 with rep=6
Accuracy No 0.7971428571428572 with rep=6
_________________________________________________________
Accuracy Yes 0.6080691642651297 with rep=7

Now we will train our NN with the **SMOTENC training dataset**. First we make the codification of the categorical values as we did with the other one:

In [ ]:
first_col=dades_train_SMOTENC.pop('UCI')
dades_train_SMOTENC.insert(0, 'UCI', first_col)
dades_train_SMOTENC=dades_train_SMOTENC.sample(frac=1)

dades_cod_train=dades_train_SMOTENC.drop('IUGR_missing',axis=1).copy()
dades_cod_test=dades_test.drop('IUGR_missing',axis=1).copy()

for df in [dades_cod_train,dades_cod_test]:
  df['UCI']=df['UCI'].apply(codUCI)
  df['Gender']=df['Gender'].apply(codGender)
  df['BW_2500']=df['BW_2500'].apply(codBW)
  df['IUGR_calc']=df['IUGR_calc'].apply(codUCI)
  for element in df.columns[11:]:
    df[element]=df[element].apply(cod)

target_train=torch.tensor(dades_cod_train['UCI'].values)
features_train=torch.tensor(dades_cod_train.drop('UCI',axis=1).values)
Dades_Tensor_train=TensorDataset(features_train,target_train)

target_test=torch.tensor(dades_cod_test['UCI'].values)
features_test=torch.tensor(dades_cod_test.drop('UCI',axis=1).values)
Dades_Tensor_test=TensorDataset(features_test,target_test)

from torch.utils.data import random_split
#train_torch,val_torch,test_torch=random_split(Dades_Tensor,[int(0.7*len(dades))+2,int(0.15*len(dades)),int(0.15*len(dades))],generator=torch.Generator().manual_seed(123))

train_torch,val_torch=random_split(Dades_Tensor_train,[int(0.8*len(Dades_Tensor_train))+1,int(0.2*len(Dades_Tensor_train))],generator=torch.Generator().manual_seed(22))
test_torch=Dades_Tensor_test

train_torch1=DataLoader(train_torch,batch_size=10,shuffle=True)
val_torch1=DataLoader(val_torch,batch_size=10,shuffle=True)
test_torch1=DataLoader(test_torch,batch_size=10,shuffle=True)

and we train and validate the Neural Network:

In [ ]:
Accuracy_yes=[]
Accuracy_no=[]

network=Network()
learn_rate=optim.Adam(network.parameters(), lr=0.0001)
for rep in range(30):
  for i in range(rep):
    for data in train_torch1:
      predictors, objectiu=data
      network.zero_grad()
      result=network(predictors.float())
      loss=F.nll_loss(result,objectiu.long())  #loss=F.cross_entropy(result,objectiu.long())  # negative log likelihood loss
      loss.backward()
      learn_rate.step()
    #print(loss)

  network.eval() #we tell the network that we are not training anymore

  correct_No=0
  correct_Yes=0
  total_No=0
  total_Yes=0

  with torch.no_grad(): #validation set to adjust the hiperparameters
    for data in val_torch1:
      predictors, objectiu=data
      result=network(predictors.float())
      for index, tensor_value in enumerate(result):
        if(objectiu[index]==1):
          total_Yes=total_Yes+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_Yes=correct_Yes+1
        if(objectiu[index]==0):
          total_No=total_No+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_No=correct_No+1

  accuracy_Yes=correct_Yes/total_Yes
  accuracy_No=correct_No/total_No

  Accuracy_yes.append(accuracy_Yes)
  Accuracy_no.append(accuracy_No)

  print('Accuracy Yes '+str(accuracy_Yes)+' with rep='+str(rep))
  print('Accuracy No '+str(accuracy_No)+' with rep='+str(rep))
  print('_________________________________________________________')

Accuracy Yes 0.0 with rep=0
Accuracy No 1.0 with rep=0
_________________________________________________________
Accuracy Yes 0.3058823529411765 with rep=1
Accuracy No 0.5294117647058824 with rep=1
_________________________________________________________
Accuracy Yes 0.35 with rep=2
Accuracy No 0.5182072829131653 with rep=2
_________________________________________________________
Accuracy Yes 0.7411764705882353 with rep=3
Accuracy No 0.30532212885154064 with rep=3
_________________________________________________________
Accuracy Yes 0.7588235294117647 with rep=4
Accuracy No 0.47619047619047616 with rep=4
_________________________________________________________
Accuracy Yes 0.9323529411764706 with rep=5
Accuracy No 0.28851540616246496 with rep=5
_________________________________________________________
Accuracy Yes 0.48823529411764705 with rep=6
Accuracy No 0.8319327731092437 with rep=6
_________________________________________________________
Accuracy Yes 0.7823529411764706 with re

Finally we will use te neural network trained by SMOTENC sample in the test dataset beacuse is the one that works better with the hyperparameter **rep**=29:

In [ ]:
correct_No=0
correct_Yes=0
total_No=0
total_Yes=0
rep=25
for data in test_torch1:
  predictors, objectiu=data
  result=network(predictors.float())
  for index, tensor_value in enumerate(result):
    if(objectiu[index]==1):
      total_Yes=total_Yes+1
      if(torch.argmax(tensor_value)==objectiu[index]):
        correct_Yes=correct_Yes+1

    if(objectiu[index]==0):
      total_No=total_No+1
      if(torch.argmax(tensor_value)==objectiu[index]):
        correct_No=correct_No+1

accuracy_Yes=correct_Yes/total_Yes
accuracy_No=correct_No/total_No

Accuracy_yes.append(accuracy_Yes)
Accuracy_no.append(accuracy_No)

print('Accuracy Yes '+str(accuracy_Yes)+' with rep='+str(rep))
print('Accuracy No '+str(accuracy_No)+' with rep='+str(rep))
print('_________________________________________________________')


Accuracy Yes 0.36 with rep=25
Accuracy No 0.8981481481481481 with rep=25
_________________________________________________________
